In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import numpy as np
import pandas as pd
import datetime
import seaborn as sns
import ogb
from tqdm import tqdm
import hiplot as hip
from copy import deepcopy
import datetime

In [3]:
import torch
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch.utils.data import Subset, TensorDataset

In [4]:
cwd = os.getcwd()
print(cwd)

/cluster/home/skyriakos/chemprop_run/git/notebooks


In [5]:
os.chdir('..')
import deepadr
from deepadr.dataset import *
from deepadr.utilities import *
from deepadr.run_workflow import *
from deepadr.chemfeatures import *
from deepadr.hyphelper import *
# from deepadr.model_gnn import GCN as testGCN
from deepadr.model_gnn_ogb import GNN, DeepAdr_SiameseTrf, ExpressionNN
# from deepadr.model_attn_siamese import *
from ogb.graphproppred import Evaluator
os.chdir(cwd)

In [6]:
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Draw

In [7]:
# from tdc.single_pred import Tox
# from tdc.multi_pred import DDI

In [8]:
rawdata_dir = '../data/raw/'
processed_dir = '../data/processed/'
up_dir = '..'

In [9]:
report_available_cuda_devices()

number of GPUs available: 8
cuda:0, name:GeForce GTX 1080 Ti
total memory available: 10.91650390625 GB
total memory allocated on device: 0.0 GB
max memory allocated on device: 0.0 GB
total memory cached on device: 0.0 GB
max memory cached  on device: 0.0 GB

cuda:1, name:GeForce GTX 1080 Ti
total memory available: 10.91650390625 GB
total memory allocated on device: 0.0 GB
max memory allocated on device: 0.0 GB
total memory cached on device: 0.0 GB
max memory cached  on device: 0.0 GB

cuda:2, name:GeForce GTX 1080 Ti
total memory available: 10.91650390625 GB
total memory allocated on device: 0.0 GB
max memory allocated on device: 0.0 GB
total memory cached on device: 0.0 GB
max memory cached  on device: 0.0 GB

cuda:3, name:GeForce GTX 1080 Ti
total memory available: 10.91650390625 GB
total memory allocated on device: 0.0 GB
max memory allocated on device: 0.0 GB
total memory cached on device: 0.0 GB
max memory cached  on device: 0.0 GB

cuda:4, name:GeForce GTX 1080 Ti
total memory av

In [10]:
n_gpu = torch.cuda.device_count()
n_gpu

8

In [11]:
device_cpu = get_device(to_gpu=False)
# device_gpu = get_device(True, index=0)

# fdtype = torch.float32

In [12]:
print("torch:", torch.__version__)
print("CUDA:", torch.version.cuda)
print(sys.version)

torch: 1.9.1
CUDA: 11.1
3.9.9 | packaged by conda-forge | (main, Dec 20 2021, 02:41:03) 
[GCC 9.4.0]


In [13]:
# print(torch.cuda.memory_summary(device=device_gpu))

### Preparing dataset 

In [14]:
score = 'total_thresh'
score_val = 4

In [15]:
# TDC Tox
# DSdataset_name = 'SynergxZloewe' #'OncoPolyPharmacology' #'DrugComb'
DSdataset_name = f'DrugComb_{score}_{score_val}' #'DrugComb'


#fname_suffix = ds_config["fname_suffix"]
similarity_types = ['chem']
kernel_option = 'sqeuclidean'
data_fname = 'data_v1'
# interact_matfname = ds_config["interact_matfname"]
# exp_iden = 'simtypeall'
# ddi_interaction_labels_pth = ds_config["ddi_interaction_labels_pth"]

# up_dir, processed_dir, DSdataset_name, data_fname

In [16]:
targetdata_dir = create_directory(os.path.join(processed_dir, DSdataset_name, data_fname))
targetdata_dir_raw = create_directory(os.path.join(targetdata_dir, "raw"))
targetdata_dir_processed = create_directory(os.path.join(targetdata_dir, "processed"))
targetdata_dir_exp = create_directory(os.path.join(targetdata_dir, "experiments"))
# # ReaderWriter.dump_data(dpartitions, os.path.join(targetdata_dir, 'data_partitions.pkl'))
print(targetdata_dir)

path_current_dir /cluster/home/skyriakos/chemprop_run/git/deepadr
path_current_dir /cluster/home/skyriakos/chemprop_run/git/deepadr
path_current_dir /cluster/home/skyriakos/chemprop_run/git/deepadr
path_current_dir /cluster/home/skyriakos/chemprop_run/git/deepadr
/cluster/home/skyriakos/chemprop_run/git/data/processed/DrugComb_total_thresh_4/data_v1


In [17]:
%%time

# Make sure to first run the "data_generation" notebook first

dataset = MoleculeDataset(root=targetdata_dir)

CPU times: user 5.08 ms, sys: 153 ms, total: 158 ms
Wall time: 159 ms


In [18]:
# print()
# print(f'Dataset: {dataset}:')
# print('====================')
# print(f'Number of graphs: {len(dataset)}')
# print(f'Number of features: {dataset.num_features}')
# print(f'Number of classes: {dataset.num_classes}')

# # data0 = dataset[0]  # Get the first graph object.

# # print()
# # print(data)
# # print('=============================================================')

# # # Gather some statistics about the first graph.
# # print(f'Number of nodes: {data.num_nodes}')
# # print(f'Number of edges: {data.num_edges}')
# # print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
# # print(f'Contains isolated nodes: {data.contains_isolated_nodes()}')
# # print(f'Contains self-loops: {data.contains_self_loops()}')
# # print(f'Is undirected: {data.is_undirected()}')

In [19]:
# data0

In [20]:
# data0.expression.shape

In [21]:
# data0.y

In [22]:
# dataset.tensors[1]

In [23]:
# used_dataset = dataset

# If you want to use a smaller subset of the dataset for testing
# smaller_dataset_len = int(len(dataset)/1)
# used_dataset = dataset[:smaller_dataset_len]

In [24]:
# type(used_dataset)

In [25]:
fold_partitions = get_stratified_partitions(dataset.data.y,
                                            num_folds=5,
                                            valid_set_portion=0.1,
                                            random_state=42)

fold_num: 0
train data
class: 0 norm count: 0.5764128559102675
class: 1 norm count: 0.4235871440897325
validation data
class: 0 norm count: 0.5764192139737991
class: 1 norm count: 0.42358078602620086
test data
class: 0 norm count: 0.5764751552795031
class: 1 norm count: 0.4235248447204969

-------------------------
fold_num: 1
train data
class: 0 norm count: 0.5764128559102675
class: 1 norm count: 0.4235871440897325
validation data
class: 0 norm count: 0.5764192139737991
class: 1 norm count: 0.42358078602620086
test data
class: 0 norm count: 0.5764751552795031
class: 1 norm count: 0.4235248447204969

-------------------------
fold_num: 2
train data
class: 0 norm count: 0.5764356969533567
class: 1 norm count: 0.4235643030466433
validation data
class: 0 norm count: 0.5764192139737991
class: 1 norm count: 0.42358078602620086
test data
class: 0 norm count: 0.5763929334109882
class: 1 norm count: 0.4236070665890118

-------------------------
fold_num: 3
train data
class: 0 norm count: 0.576

In [26]:
# train_val_test_frac = [0.7, 0.1, 0.2]
# assert sum(train_val_test_frac) == 1

# torch.manual_seed(42)
# used_dataset = used_dataset.shuffle()

# num_train = round(train_val_test_frac[0] * len(used_dataset)) 
# num_trainval = round((train_val_test_frac[0]+train_val_test_frac[1]) * len(used_dataset))

# train_dataset = used_dataset[:num_train]
# val_dataset = used_dataset[num_train:num_trainval]
# test_dataset = used_dataset[num_trainval:]

In [27]:
# train_dataset = Subset(used_dataset, fold_partitions[0]['train'])
# val_dataset = Subset(used_dataset, fold_partitions[0]['validation'])
# test_dataset = Subset(used_dataset, fold_partitions[0]['test'])

In [28]:
# print(f'Number of training graphs: {len(train_dataset)}')
# print(f'Number of val graphs: {len(val_dataset)}')
# print(f'Number of test graphs: {len(test_dataset)}')

In [29]:
print("Number of training graphs: "+ str(len(fold_partitions[0]['train'])))
print("Number of validation graphs: "+ str(len(fold_partitions[0]['validation'])))
print("Number of testing graphs: "+ str(len(fold_partitions[0]['test'])))


Number of training graphs: 18544
Number of validation graphs: 2061
Number of testing graphs: 5152


In [30]:
# print(dataset.num_classes)

In [31]:
# training params total_thresh 4
tp = {
    "batch_size" : 300,
    "num_epochs" : 20,
    
    "emb_dim" : 300,
    "gnn_type" : "gatv2",
    "num_layer" : 5,
    "graph_pooling" : "mean", #attention
    
    "input_embed_dim" : None,
    "gene_embed_dim": 1,
    "num_attn_heads" : 2,
    "num_transformer_units" : 1,
    "p_dropout" : 0.3,
#     "nonlin_func" : nn.ReLU(),
    "mlp_embed_factor" : 2,
    "pooling_mode" : 'attn',
    "dist_opt" : 'cosine',

    "base_lr" : 3e-5, #3e-4
    "max_lr_mul": 15,
    "l2_reg" : 1e-7,
    "loss_w" : 0.95,
    "margin_v" : 1.,

    "expression_dim" : 64,
    "expression_input_size" : 908,
    "exp_H1" : 500,
    "exp_H2" : 400
}

In [32]:
# # training params total_thresh 3
# tp = {
#     "batch_size" : 300,
#     "num_epochs" : 200,
    
#     "emb_dim" : 300,
#     "gnn_type" : "gatv2",
#     "num_layer" : 5,
#     "graph_pooling" : "mean", #attention
    
#     "input_embed_dim" : None,
#     "gene_embed_dim": 1,
#     "num_attn_heads" : 2,
#     "num_transformer_units" : 1,
#     "p_dropout" : 0.3,
# #     "nonlin_func" : nn.ReLU(),
#     "mlp_embed_factor" : 2,
#     "pooling_mode" : 'attn',
#     "dist_opt" : 'cosine',

#     "base_lr" : 3e-5, #3e-4
#     "max_lr_mul": 10,
#     "l2_reg" : 1e-8,
#     "loss_w" : 0.3,
#     "margin_v" : 1.,

#     "expression_dim" : 64,
#     "expression_input_size" : 908,
#     "exp_H1" : 500,
#     "exp_H2" : 400
# }

In [33]:
# targetdata_dir_exp = create_directory(os.path.join(targetdata_dir, "experiments"))
# targetdata_dir_exp

In [34]:
# device_gpu = get_device(True, index=0)
# exp_dir = create_directory(os.path.join(targetdata_dir_exp, "exp0"))
# tphp = generate_tp_hp(tp, hyperparam_space[0], hp_names)
# partition = fold_partitions[0]

In [35]:
# run_exp(tphp, device_gpu, exp_dir,partition=fold_partitions[0], used_dataset=used_dataset)

In [36]:
def spawn_q_process(q_process):
    print(">>> spawning hyperparam search process")
    q_process.start()
    
def join_q_process(q_process):
    q_process.join()
    print("<<< joined hyperparam search process")
    
def create_q_process(queue, used_dataset, gpu_num, tphp, exp_dir, partition): #
#     fold_gpu_map = {0:gpu_num}
    return mp.Process(target=deepadr.hyphelper.run_exp, args=(queue, used_dataset, gpu_num, tphp, exp_dir, partition)) #

In [37]:
import torch.multiprocessing as mp
mp.set_start_method("spawn", force=True)

queue = mp.Queue()
q_processes = []

# partition = fold_partitions[0]
time_stamp = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')

print("Start: " + time_stamp)

for q_i in range(min(n_gpu, len(fold_partitions))):
#     device_gpu = get_device(True, index=q_i)
    partition = fold_partitions[q_i]
    exp_dir = create_directory(os.path.join(targetdata_dir_exp, "fold_"+str(q_i)+"_"+time_stamp))
    create_directory(os.path.join(exp_dir, "predictions"))

#     tphp = generate_tp_hp(tp, hyperparam_space[q_i], hp_names)
    
    q_process = create_q_process(queue, dataset, q_i, tp, exp_dir, partition)
    q_processes.append(q_process)
    spawn_q_process(q_process)

spawned_processes = n_gpu
    
# for q_i in range(len(hyperparam_space)):
for q_i in range(min(n_gpu, len(fold_partitions))):
    join_q_process(q_processes[q_i])
    released_gpu_num = queue.get()
    print("released_gpu_num:", released_gpu_num)
#     if(spawned_processes < len(hyperparam_space)):
# #         device_gpu = get_device(True, index=q_i)
#         time_stamp = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
#         exp_dir = create_directory(os.path.join(targetdata_dir_exp, "exp_"+str(q_i)+"_"+time_stamp))
#         tphp = generate_tp_hp(tp, hyperparam_space[q_i], hp_names)

#         q_process = create_q_process(queue, used_dataset, released_gpu_num, tphp, exp_dir, partition)
#         q_processes.append(q_process)
#         spawn_q_process(q_process)
#         spawned_processes = spawned_processes + 1

Start: 2022-02-26_13-44-37
path_current_dir /cluster/home/skyriakos/chemprop_run/git/deepadr
path_current_dir /cluster/home/skyriakos/chemprop_run/git/deepadr
>>> spawning hyperparam search process
path_current_dir /cluster/home/skyriakos/chemprop_run/git/deepadr
path_current_dir /cluster/home/skyriakos/chemprop_run/git/deepadr
>>> spawning hyperparam search process


Iteration:   0%|          | 0/62 [00:00<?, ?it/s]

gpu: cuda:0
GeneEmbAttention pooling, gene_embed_dim: 1
updated
num classes: 2
=====Epoch 0
Training...
path_current_dir /cluster/home/skyriakos/chemprop_run/git/deepadr
path_current_dir /cluster/home/skyriakos/chemprop_run/git/deepadr
>>> spawning hyperparam search process


Iteration:   0%|          | 0/62 [00:00<?, ?it/s].26s/it]

gpu: cuda:1
GeneEmbAttention pooling, gene_embed_dim: 1
updated
num classes: 2
=====Epoch 0
Training...
path_current_dir /cluster/home/skyriakos/chemprop_run/git/deepadr
path_current_dir /cluster/home/skyriakos/chemprop_run/git/deepadr
>>> spawning hyperparam search process


...

{'Train':  best_epoch_indx:19
 auc:0.8538867475805333 
 apur:0.813525781625641 
 f1:0.7431852599697123 
 precision:0.7367696734642812 
 recall:0.7497135582431572 
, 'Validation':  best_epoch_indx:19
 auc:0.8429334390101858 
 apur:0.8020014204700263 
 f1:0.7280949689089881 
 precision:0.71875 
 recall:0.7376861397479955 
, 'Test':  best_epoch_indx:19
 auc:0.8472183074785307 
 apur:0.8104055130350197 
 f1:0.7258027258027259 
 precision:0.7317186772240335 
 recall:0.7199816681943172 
}
Finished training!


Iteration:   0%|          | 0/7 [00:00<?, ?it/s]

{'Train':  best_epoch_indx:19
 auc:0.8536849313355552 
 apur:0.8171114793083776 
 f1:0.7402985074626866 
 precision:0.7236474164133738 
 recall:0.757733927434755 
, 'Validation':  best_epoch_indx:19
 auc:0.8347989247187413 
 apur:0.7893223282258983 
 f1:0.7220982142857142 
 precision:0.7040261153427638 
 recall:0.7411225658648339 
, 'Test':  best_epoch_indx:19
 auc:0.8485244254794194 
 apur:0.8090574062305417 
 f1:0.7355650415636936 
 precision:0.7214631996474218 
 recall:0.7502291475710358 
}
Finished training!


Iteration: 100%|██████████| 7/7 [00:00<00:00, 15.66it/s]


{'Train':  best_epoch_indx:19
 auc:0.8579939311622787 
 apur:0.823763206085669 
 f1:0.7463971775399151 
 precision:0.703766350924673 
 recall:0.7945257797581159 
, 'Validation':  best_epoch_indx:19
 auc:0.8358638889853094 
 apur:0.7949261263322005 
 f1:0.7242666666666667 
 precision:0.6776447105788423 
 recall:0.7777777777777778 
, 'Test':  best_epoch_indx:19
 auc:0.8479498200006854 
 apur:0.8078911661840362 
 f1:0.73840206185567 
 precision:0.6948261924009701 
 recall:0.7878093492208983 
}
Finished training!
<<< joined hyperparam search process
released_gpu_num: 4
<<< joined hyperparam search process
released_gpu_num: 2
<<< joined hyperparam search process
released_gpu_num: 3


In [38]:
print("End: " + datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S'))


End: 2022-02-26_13-51-57


### Hyper Parameters

In [42]:
time_stamp = "2022-02-26_13-44-37"

import glob
exp_dirs = glob.glob(targetdata_dir_exp+"/fold_*_"+time_stamp)
len(exp_dirs)

5

In [43]:
folds = []

for edir in exp_dirs:
    fold = edir.split("/")[-1].split('_')[1]
    df_curves = pd.read_csv(edir + "/curves.csv")
#     le = df_curves.tail(1)
#     folds.append([fold, le.test_aupr.values[0], le.test_auc.values[0]])
    df_curves["fold"] = [fold] * len(df_curves)
    folds.append(df_curves)

In [44]:
df_folds = pd.concat(folds)
df_folds

,epoch,train_aupr,valid_aupr,test_aupr,train_auc,valid_auc,test_auc,fold
0,0,0.653898,0.635664,0.643471,0.703354,0.689977,0.698164,0
1,1,0.708209,0.683457,0.698030,0.768811,0.753978,0.766734,0
2,2,0.731741,0.707038,0.716086,0.792020,0.775331,0.783754,0
3,3,0.763432,0.738274,0.755569,0.811440,0.796185,0.808683,0
4,4,0.766819,0.744528,0.756065,0.819881,0.805024,0.812908,0
...,...,...,...,...,...,...,...,...
15,15,0.799132,0.783491,0.793792,0.843207,0.832285,0.833529,4
16,16,0.807471,0.794695,0.807116,0.849256,0.838631,0.842821,4
17,17,0.811031,0.801216,0.806963,0.850913,0.839673,0.843451,4
18,18,0.812000,0.799301,0.808463,0.851822,0.839726,0.844592,4


In [45]:
df_folds_epoch = df_folds.groupby("epoch")[["test_aupr", "test_auc"]].mean()
df_folds_epoch["Fscore"] = df_folds_epoch.apply(lambda x: (2*x[0]*x[1])/(x[0]+x[1]), axis=1) # harmonic mean of AUC, AUPR
df_folds_epoch

,test_aupr,test_auc,Fscore
epoch,,,
0,0.648071,0.708316,0.676856
1,0.715236,0.771278,0.742201
2,0.733511,0.790470,0.760926
3,0.751610,0.803790,0.776825
4,0.761017,0.812080,0.785720
5,0.773550,0.822303,0.797182
6,0.782840,0.827771,0.804679
7,0.788285,0.831970,0.809539
8,0.793409,0.836053,0.814173


In [46]:
best_epoch = df_folds_epoch.Fscore.idxmax()
df_folds_epoch.loc[best_epoch]

test_aupr    0.813289
test_auc     0.851352
Fscore       0.831886
Name: 19, dtype: float64

### Attention Scores

In [47]:
l_attn = []

for edir in exp_dirs:
    attn_scores = pd.read_csv(exp_dir + "/fattn_w_scores_e_ids_test.csv")
    l_attn.append(attn_scores)